In [49]:
import tensorflow as tf

In [63]:
def mnist_reader(part):
    for i in range(10):
        with open('Part3_{}_{}.csv'.format(i, part)) as f:
            for line in f:
                yield line.strip(), i

def mnist_decoder(csv_line, label):
    FIELD_DEFAULTS = [[0.0]]*(28*28)
    with tf.variable_scope('DataSource'):
        fields = tf.decode_csv(csv_line, FIELD_DEFAULTS)
        im = tf.stack(fields)
        im = tf.reshape(im, (28, 28))
        return im, tf.one_hot(label, depth=10)

In [64]:
tf.reset_default_graph()

with tf.variable_scope('DataSource'):
    dataset = tf.data.Dataset.from_generator(lambda: mnist_reader('Train'),
        # csv_line, label
        (tf.string, tf.int32),
        (tf.TensorShape([]), tf.TensorShape([])
    )).map(mnist_decoder, num_parallel_calls=2) \
      .batch(128) \
      .prefetch(1) \
      .repeat(50)

    iter_handle = tf.placeholder(tf.string, shape=[])
    data_iterator = tf.data.Iterator.from_string_handle(iter_handle, dataset.output_types, dataset.output_shapes)
    train_iterator = dataset.make_one_shot_iterator()
    #         val_iterator = dataset_val.make_initializable_iterator()
    images, onehot_labels = data_iterator.get_next()

with tf.variable_scope('Initializer'):
    init_global = tf.global_variables_initializer()
    init_local = tf.local_variables_initializer()

In [65]:
with tf.Session() as sess:
    sess.run(init_global)
    train_handle = sess.run(train_iterator.string_handle())
    sess.run(init_local)
    for i in range(5000):
        sess.run([images], feed_dict={iter_handle: train_handle})

KeyboardInterrupt: 